<a href="https://colab.research.google.com/github/aaronjoseph/NLP_Boiler_Plate/blob/main/Sentiment_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install mlflow
#Load the libraries
from mlflow import log_metric, log_param, log_artifacts
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
import os
from tqdm import tqdm
import collections, numpy
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
tqdm.pandas()
from sklearn.model_selection import train_test_split,KFold
nltk.download('stopwords')
! pip install neptune-client==0.4.132
import neptune
neptune.init(
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiOGI1M2QwMmItZGQyMi00MTBmLThhZTktZDY0OThjYmZlMGMyIn0=",
    project_qualified_name="aaronjosephmathew/sandbox"
)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Project(aaronjosephmathew/sandbox)

In [ ]:
# 
sentiment_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',header=None,error_bad_lines=False,engine='python')
#sentiment_data = sentiment_data.sample(n=1_00_000,replace=True)

In [ ]:
# Data Cleaning Process
def preprocess(text):
  text=text.lower()
  # remove hyperlinks
  text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)
  text = re.sub(r'http?:\/\/.*[\r\n]*', '', text)
  #remove hashtag sign
  text=re.sub(r"#","",text)   
  #remove mentions
  text = re.sub(r"(?:\@)\w+", '', text)
  #remove non ascii chars
  text=text.encode("ascii",errors="ignore").decode()
  #remove some puncts (except . ! ?)
  text=re.sub(r'[:"#$%&\*+,-/:;<=>@\\^_`{|}~]+','',text)
  text=re.sub(r'[!]+','!',text)
  text=re.sub(r'[?]+','?',text)
  text=re.sub(r'[.]+','.',text)
  text=re.sub(r"'","",text)
  text=re.sub(r"\(","",text)
  text=re.sub(r"\)","",text)    
  text=" ".join(text.split())
  return text

def simple_stemmer(text):
  ps=nltk.porter.PorterStemmer()
  text= ' '.join([ps.stem(word) for word in text.split()])
  return text

#removing the stopwords
def remove_stopwords(text, is_lower_case=False):
  #set stopwords to english
  stop=set(stopwords.words('english'))
  # Tokenization
  tokenizer=ToktokTokenizer()
  #Setting English stopwords
  stopword_list=nltk.corpus.stopwords.words('english')
  tokens = tokenizer.tokenize(text)
  tokens = [token.strip() for token in tokens]
  if is_lower_case:
      filtered_tokens = [token for token in tokens if token not in stopword_list]
  else:
      filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
  filtered_text = ' '.join(filtered_tokens)    
  return filtered_text

In [ ]:
#Apply function
sentiment_data['clean']=sentiment_data[5].progress_apply(preprocess)
sentiment_data['stem']=sentiment_data['clean'].progress_apply(simple_stemmer)
sentiment_data['token'] = sentiment_data['stem'].progress_apply(remove_stopwords) 

100%|██████████| 100000/100000 [00:38<00:00, 2596.52it/s]


In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, auc, roc_curve, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()
sentiment_data['output'] = le.fit_transform(sentiment_data[0])
# Splitting the data - Into train_test
X = sentiment_data['token']
y = sentiment_data['output']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42) 

In [ ]:
# Count Vectorizer for Bag of words
cv=CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,3))
#transformed train reviews
cv_train_reviews=cv.fit_transform(X_train)
#transformed test reviews
cv_test_reviews=cv.transform(X_test)
# TFIDF
tv=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))
#transformed train reviews
tv_train_reviews=tv.fit_transform(X_train)
#transformed test reviews
tv_test_reviews=tv.transform(X_test)

In [ ]:
# Develop XGBoost Model [Classifier/Regressor]
xgb_cv = XGBClassifier(max_depth=6,n_estimators=1000).fit(cv_train_reviews,y_train)
xgb_tfidf = XGBClassifier(max_depth=6,n_estimators=1000).fit(tv_train_reviews,y_train)

# Prediction
prediction_cv = xgb_cv.predict(cv_test_reviews)
prediction_tfidf = xgb_tfidf.predict(tv_test_reviews) 

In [ ]:
def scoring(y_pred,y_true,experiment_Name="Standard"):
  
  # Accuracy 
  a_c = accuracy_score(y_true, y_pred)
  print('Accuracy Score', a_c)

  # Recall & Precision  
  p_s = precision_score(y_true, y_pred)
  r_s = recall_score(y_true, y_pred)
  print('Precision Score',p_s)
  print('Recall Score', r_s)
  
  # F1 Score
  f1_s = f1_score(y_true, y_pred)
  print('F1 Score', f1_s)

  neptune.create_experiment(experiment_Name)
  neptune.log_metric('accuracy_score', a_c)
  neptune.log_metric('precision_score', p_s)
  neptune.log_metric('recall_score', r_s)
  neptune.log_metric('f1_score', f1_s)
  neptune.stop()

In [ ]:
print("Printing Metrics for XGboost Count Vectorizer")
scoring(prediction_cv,y_test,"XGBoost Count Vectorizer")
print("="*120,"\nPrinting Metrics for XGboost TFIDF")
scoring(prediction_tfidf,y_test,"XGBoost TFIDF")

Printing Metrics for XGboost Count Vectorizer
Accuracy Score 0.49815
Precision Score 1.0
Recall Score 0.49815
F1 Score 0.6650201915696026
https://ui.neptune.ai/aaronjosephmathew/sandbox/e/SAN-5
Printing Metrics for XGboost TFIDF
Accuracy Score 0.49815
Precision Score 1.0
Recall Score 0.49815
F1 Score 0.6650201915696026
https://ui.neptune.ai/aaronjosephmathew/sandbox/e/SAN-6


Parameters

**Default parameters**

max_depth=3,
learning_rate=0.1, 
n_estimators=100, 
verbosity=1, 
silent=None, 
objective='reg:squarederror', 
booster='gbtree', 
n_jobs=1, 
nthread=None, 
gamma=0, 
min_child_weight=1, 
max_delta_step=0, 
subsample=1, 
colsample_bytree=1, 
colsample_bylevel=1, 
colsample_bynode=1, 
reg_alpha=0, 
reg_lambda=1,
scale_pos_weight=1, 
base_score=0.5, 
random_state=0, 
seed=None,
missing=None, 
importance_type='gain'

**Explanation of relevant parameters for this kernel.**

* booster: Select the type of model to run at each iteration
* gbtree: tree-based models
* gblinear: linear models
* nthread: default to maximum number of threads available if not set
* objective: This defines the loss function to be minimized

**Parameters for controlling speed**

subsample: Denotes the fraction of observations to be randomly samples for each tree

colsample_bytree: Subsample ratio of columns when constructing each tree.

n_estimators: Number of trees to fit.
Important parameters which control overfiting

learning_rate: Makes the model more robust by shrinking the weights on each step

max_depth: The maximum depth of a tree.

min_child_weight: Defines the minimum sum of weights of all observations 
required in a child.

Tuning the hyper-parameters

GridSearchCV params:

estimator: estimator object

param_grid : dict or list of dictionaries

scoring: A single string or a callable to evaluate the predictions on the test 
set. If None, the estimator’s score method is used.

https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
n_jobs: Number of jobs to run in parallel. None means. -1 means using all processors.

cv: cross-validation, None, to use the default 3-fold cross validation. Integer, to specify the number of folds in a (Stratified)KFold.


In [ ]:
# Hyper-parameter Tuning using Grid Search to get better score
# Defination for Hyperparameter tuning
#XGBoost hyper-parameter tuning
def hyperParameterTuning(X_train, y_train):
    param_tuning = {
        'learning_rate': [0.01, 0.1],
        'max_depth': [3, 5, 7, 10],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.5, 0.7],
        'colsample_bytree': [0.5, 0.7],
        'n_estimators' : [100, 200, 500],
        'objective': ['reg:squarederror']
    }
    xgb_model = XGBClassifier()

    gsearch = GridSearchCV(estimator = xgb_model,
                           param_grid = param_tuning,                        
                           cv = 5,
                           n_jobs = -1,
                           verbose = 1)
    gsearch.fit(X_train,y_train)
    return gsearch.best_params_

In [ ]:
cv_train_reviews

<80000x514305 sparse matrix of type '<class 'numpy.int64'>'
	with 514305 stored elements in Compressed Sparse Row format>

In [ ]:
# Hyperparameter tuning for Count Vectorizer Model
# hyperParameterTuning(cv_train_reviews,y_train)
"""
Parameters
{'colsample_bytree': 0.5,
 'learning_rate': 0.01,
 'max_depth': 3,
 'min_child_weight': 1,
 'n_estimators': 100,
 'objective': 'reg:squarederror',
 'subsample': 0.5}
"""

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:  7.7min
[Parallel(n_jobs=-1)]: Done 370 tasks      | elapsed: 31.0min
[Parallel(n_jobs=-1)]: Done 720 tasks      | elapsed: 62.0min
[Parallel(n_jobs=-1)]: Done 1170 tasks      | elapsed: 112.9min
[Parallel(n_jobs=-1)]: Done 1440 out of 1440 | elapsed: 153.6min finished


{'colsample_bytree': 0.5,
 'learning_rate': 0.01,
 'max_depth': 3,
 'min_child_weight': 1,
 'n_estimators': 100,
 'objective': 'reg:squarederror',
 'subsample': 0.5}

In [ ]:
xgb_model_cv = XGBClassifier(
        objective = 'reg:squarederror',
        colsample_bytree = 0.5,
        learning_rate = 0.01,
        max_depth = 3,
        min_child_weight = 1,
        n_estimators = 100,
        subsample = 0.5)

xgb_model_cv.fit(cv_train_reviews,y_train, verbose=False)

y_pred_cv = xgb_model_cv.predict(cv_test_reviews)

In [ ]:
print("Printing Metrics for XGboost Count Vectorizer")
scoring(y_pred_cv,y_test,"XGBoost Count Vectorizer")

Printing Metrics for XGboost Count Vectorizer
Accuracy Score 0.49845
Precision Score 0.9997992572518318
Recall Score 0.4982991495747874
F1 Score 0.6651086702500584
https://ui.neptune.ai/aaronjosephmathew/sandbox/e/SAN-7


In [ ]:
# Code for LGBM Classifier
model_lgb = LGBMClassifier(num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)
model_lgb.fit(cv_train_reviews,y_train)
model_lgb_pred = model_lgb.predict(cv_test_reviews)
print("Printing Metrics for LGBM Count Vectorizer")
scoring(model_lgb_pred,y_test,"LGBM Count Vectorizer")

Model Stacking

In [ ]:
def get_oof(clf, x_train, y_train, x_test):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf):
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [ ]:
# Random Forest parameters
rf_params = {
    'n_jobs': -1,
    'n_estimators': 500,
     'warm_start': True, 
     #'max_features': 0.2,
    'max_depth': 6,
    'min_samples_leaf': 2,
    'max_features' : 'sqrt',
    'verbose': 0
}

r_clf = RandomForestClassifier(**rf_params)
r_clf.fit(cv_train_reviews,y_train)
r_pred = r_clf.predict(cv_test_reviews)

print("Printing Metrics for Random Forest Count Vectorizer")
scoring(r_pred,y_test,"Random Forest Count Vectorizer")

Printing Metrics for Random Forest Count Vectorizer
Accuracy Score 0.49815
Precision Score 1.0
Recall Score 0.49815
F1 Score 0.6650201915696026
https://ui.neptune.ai/aaronjosephmathew/sandbox/e/SAN-8


In [ ]:
# Extra Trees Parameters
et_params = {
    'n_jobs': -1,
    'n_estimators':500,
    #'max_features': 0.5,
    'max_depth': 8,
    'min_samples_leaf': 2,
    'verbose': 0
}

e_trees = ExtraTreesClassifier(**et_params)
e_trees.fit(cv_train_reviews,y_train)
e_trees_pred = e_trees.predict(cv_test_reviews)

print("Printing Metrics for Extra Trees Count Vectorizer")
scoring(e_trees_pred,y_test,"Extra Trees Count Vectorizer")


Printing Metrics for Extra Trees Count Vectorizer
Accuracy Score 0.49815
Precision Score 0.49815
Recall Score 1.0
F1 Score 0.6650201915696026
https://ui.neptune.ai/aaronjosephmathew/sandbox/e/SAN-10


In [ ]:
# AdaBoost parameters
ada_params = {
    'n_estimators': 500,
    'learning_rate' : 0.75
}

# Gradient Boosting parameters
gb_params = {
    'n_estimators': 500,
     #'max_features': 0.2,
    'max_depth': 5,
    'min_samples_leaf': 2,
    'verbose': 0
}

# Support Vector Classifier parameters 
svc_params = {
    'kernel' : 'linear',
    'C' : 0.025
    }

ada_clf = AdaBoostClassifier(**ada_params)
ada_clf.fit(cv_train_reviews,y_train)
ada_clf_pred = ada_clf.predict(cv_test_reviews)
print("Printing Metrics for Ada Boost Count Vectorizer")
scoring(ada_clf_pred,y_test,"Ada Boost Count Vectorizer")


gb_clf = GradientBoostingClassifier(**gb_params)
gb_clf.fit(cv_train_reviews,y_train)
gb_clf_pred = gb_clf.predict(cv_test_reviews)
print("Printing Metrics for Gra Boost Count Vectorizer")
scoring(ada_clf_pred,y_test,"Gra Boost Count Vectorizer")

SVC_clf = SVC(**svc_params)
SVC_clf.fit(cv_train_reviews,y_train)
SVC_clf_pred = SVC_clf.predict(cv_test_reviews)
print("Printing Metrics for SVM Count Vectorizer")
scoring(SVC_clf_pred,y_test,"SVM Count Vectorizer")


Printing Metrics for Ada Boost Count Vectorizer
Accuracy Score 0.4989
Precision Score 0.498523153942428
Recall Score 0.9994981431295794
F1 Score 0.6652414990981361
https://ui.neptune.ai/aaronjosephmathew/sandbox/e/SAN-11
Printing Metrics for Gra Boost Count Vectorizer
Accuracy Score 0.4989
Precision Score 0.498523153942428
Recall Score 0.9994981431295794
F1 Score 0.6652414990981361
https://ui.neptune.ai/aaronjosephmathew/sandbox/e/SAN-12
Printing Metrics for SVM Count Vectorizer
Accuracy Score 0.50275
Precision Score 0.5004520795660036
Recall Score 1.0
F1 Score 0.6670683940946067
https://ui.neptune.ai/aaronjosephmathew/sandbox/e/SAN-13
